In [1]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(0)

2022-12-20 14:15:44.981470: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 14:15:45.064159: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-20 14:15:45.064175: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-20 14:15:45.602586: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
url = 'https://raw.githubusercontent.com/igomezv/nnogada/main/data/jla.csv'
df=pd.read_csv(url)
N = len(df.values)
randomize = np.random.permutation(N)
data = df.values[randomize]
N = len(df.values)
z = data[:,0] 
y = data[:,1:3] ### coge el resto de variables a predecir 
y[:,1] = y[:,1]**2+data[:,2]
np.shape(y)
df

zcmb         mb       dmb    errors
0    0.503084  23.001698  0.088031  0.120219
1    0.580724  23.573937  0.090132  0.121435
2    0.494795  22.960139  0.088110  0.089552
3    0.345928  22.398137  0.087263  0.119729
4    0.677662  24.078115  0.098356  0.100088
..        ...        ...       ...       ...
735  0.027064  16.504006  0.141685  0.185894
736  0.025468  15.797848  0.143429  0.193666
737  0.023810  15.895501  0.144315  0.184957
738  0.023867  16.068268  0.144350  0.175960
739  0.022068  15.718540  0.144685  0.160803

[740 rows x 4 columns]

In [3]:
dmag =df["dmb"]

df2=df['errors']+df['dmb']**2

In [4]:
scalerz = StandardScaler()
scalerz.fit(z.reshape(-1,1))
z = scalerz.transform(z.reshape(-1,1))

In [5]:
### Modificar para incluir phanteon como test
split = 0.75
ntrain = int(split * len(z))
indx = [ntrain]
X_train, X_test = np.split(z, indx)
Y_train, Y_test = np.split(y, indx)
np.shape(X_train), np.shape(X_test), Y_train.shape, Y_test.shape

((555, 1), (185, 1), (555, 2), (185, 2))

In [6]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.0,
                                   patience=200,
                                   restore_best_weights=True, verbose=False)
                                   ]

n_cols = 1

In [7]:
epochs=200

In [8]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Dense(int(X_train.shape[1])))
    
    for i in range(hparams['HP_LAYERS']):        
        model.add(Dense(hparams['HP_NUM_UNITS'], activation='relu'))
    model.add(Dense(2, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams['HP_LEARNING'], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    train = model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=hparams['HP_BATCHSIZE'], shuffle=False, verbose=False)

    _, loss = model.evaluate(X_test, Y_test)
    
    return model, loss, train.history

# Genetic results

## B

### P_mutation = 0.2, P_crossover = 0.5

In [9]:
# 3	200	0.0001	4
hparams_gena1 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 4	50	0.0001	4
hparams_gena2 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 50, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 1	100	0.001	4
hparams_gena3 = {'HP_LAYERS': 1, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.001}
# 1	100	0.001	4
hparams_gena4 = {'HP_LAYERS': 1, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.001}
# 3	100	0.0001	4
hparams_gena5 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 1	100	0.001	4
hparams_gena6 = {'HP_LAYERS': 1, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.001}
# 3	100	0.0001	4
hparams_gena7 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 3 	100 	0.0001 	4
hparams_gena8 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 1	100	0.001	4
hparams_gena9 = {'HP_LAYERS': 1, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.001}
# 4	100	0.0001	4
hparams_gena10 = {'HP_LAYERS': 4,'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}

In [15]:
models_gena = []
mse_gena = []
mae_gena = []
r2_gena = []
histories_gena = []
hparams_gena = [hparams_gena1, hparams_gena2, hparams_gena3,hparams_gena4,hparams_gena5,hparams_gena6,hparams_gena7,
                hparams_gena8,hparams_gena9, hparams_gena10]

for i in range(10):
    model, loss_test, history = train_test_model(hparams_gena[i])
    models_gena.append(model)
    mse_gena.append(loss_test)
    histories_gena.append(history)
    y = model.predict(X_test)
    r2_gena.append(r2_score(y, Y_test))
    mae_gena.append(mean_absolute_error(y, Y_test))
    
for i in range(10):
    print("Run {}: {:.4f}\t{:.4f}\t{:.4f}".format(i+1, mse_gena[i], mae_gena[i], r2_gena[i]))

6/6 [==============================] - 0s 772us/step
Run 1: 0.0313	0.0996	0.6414
Run 2: 0.0320	0.0997	0.6893
Run 3: 0.0331	0.1016	0.6060
Run 4: 0.0328	0.1028	0.4797
Run 5: 0.0330	0.1027	0.6823
Run 6: 0.0322	0.1021	0.4873
Run 7: 0.0318	0.1000	0.6242
Run 8: 0.0338	0.1024	0.6372
Run 9: 0.0347	0.1027	0.6965
Run 10: 0.0326	0.1047	0.6929
